In [1]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

import sys
sys.path.insert(0, '../')
coralme_dir = '../'#'/home/chris/zuniga/coralme/'
sys.path.insert(0, coralme_dir)

import importlib
from cobra.core import Reaction, Metabolite
import coralme
import coralme.solver.solver
import coralme.builder.main
import coralme.core.model
from coralme.util.helpers import get_nlp, optimize
import os
import pandas as pd
import json
import cobra
import numpy as np
import matplotlib.pyplot as plt
from coralme.util.helpers import get_nlp, optimize
import pickle

from coralme.builder.main import MEBuilder

# load ME and M models
base_dir = os.path.join(coralme_dir, 'species_files', 'Pseudomonas_files')
path = os.path.join(base_dir, 'individual_species', 'CP065866', 'outputs', 'MEModel-step3-Pseudomonas_aeruginosa_TJ2019-017-TS.pkl')
ME_model = coralme.io.pickle.load_pickle_me_model(path)


# Create a sink reaction
def add_sink_reaction_to_me(me_model, metabolite_id, lb = -1e-13, ub = -1e-13):
    """
    Add a sink reaction to a coralME model
    
    Parameters:
    -----------
    me_model : coralme.core.MEModel
        The ME model to add the sink to
    metabolite_id : str
        The ID of the metabolite to create a sink for
    lb : float
        Lower bound of the sink reaction
    ub : float
        Upper bound of the sink reaction
    """
    from coralme.core.reaction import MEReaction
    
    # Create the sink reaction
    sink_id = f'SINK_{metabolite_id}'
    sink_reaction = MEReaction(sink_id)
    sink_reaction.name = f'Sink reaction for {metabolite_id}'
    
    # Get the metabolite
    metabolite = me_model.metabolites.get_by_id(metabolite_id)
    
    # Add the metabolite to the reaction
    sink_reaction.add_metabolites({metabolite: -1})
    
    # Set the bounds
    sink_reaction.lower_bound = lb
    sink_reaction.upper_bound = ub
    
    # Add to model
    me_model.add_reactions([sink_reaction])
    
    return sink_reaction

In [ ]:
# run through each metabolite
path = os.path.join(coralme_dir, 'data', 'metab_to_sol.pkl')

# let's add all of the reactions, turned off, to the model
for metab in ME_model.metabolites:
    if 'MONOMER' not in metab.id:
        continue
    # I want to create a sink reaction
    sink_reaction = add_sink_reaction_to_me(ME_model, metab.id, lb = 0, ub = 0)

nlp = get_nlp(ME_model)
rxn_index_dct = {r.id : ME_model.reactions.index(r) for r in ME_model.reactions}
met_index_dct = {m.id : ME_model.metabolites.index(m) for m in ME_model.metabolites}

# load in solution dictionary
basis = None
if os.path.exists(path):
    pickle_in = open(path, 'rb')
    metab_to_sol = pickle.load(pickle_in)
    pickle_in.close()
else:
    metab_to_sol = {}
    sol, basis = optimize(rxn_index_dct,met_index_dct,nlp, max_mu = 0.5, min_mu = 0.05, maxIter = 20,
		tolerance = 1e-4, precision = 'quad', verbose = True, basis = None)
    metab_to_sol.update({'base': sol})

# metabolite by metabolite, turn them on, run, turn them off
for metab in ME_model.metabolites:
    if 'MONOMER' not in metab.id:
        continue
    if metab.id in metab_to_sol:
        continue

    # set bounds to force small run
    nlp.xl[rxn_index_dct['SINK_'+metab.id]] = lambda x : -1e-10
    nlp.xu[rxn_index_dct['SINK_'+metab.id]] = lambda x : -1e-10

    # Run the model, save results
    sol, basis = optimize(rxn_index_dct, met_index_dct, nlp, max_mu = 0.5, min_mu = 0.05, maxIter = 20,
		tolerance = 1e-4, precision = 'quad', verbose = True, basis = basis)
    metab_to_sol.update({metab.id : sol})
    pickle_out = open(path, 'wb')
    pickle.dump(metab_to_sol, pickle_out)
    pickle_out.close()

    # reset reaction bounds
    nlp.xl[rxn_index_dct['SINK_'+metab.id]] = lambda x : 0
    nlp.xu[rxn_index_dct['SINK_'+metab.id]] = lambda x : 0

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
